In [9]:
import pandas as pd
import numpy as np
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score
from keras.layers import *

In [2]:
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/bagging/*')):
    print('file path',feat)
    a,b,c = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(c)
# load y
train = pd.read_csv("../input/new_data/train_set.csv")
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/bagging/rnn_model_bagging1.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging2.pkl
(56590, 19) (56590, 19)
file path ../feature/bagging/rnn_model_bagging3.pkl
(56590, 19) (56590, 19)
(56590, 57)


In [3]:
train_y=b

In [4]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [5]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [11]:
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
def get_nn():
    cv1_input = Input(shape=(57,), dtype='float32')
#     merged = concatenate([cv1_input])
    merged = Dense(512,activation='relu')(cv1_input)
    merged = Dropout(0.2)(merged)
    x = Dense(19, activation="softmax")(merged)
    model = Model(inputs=cv1_input, outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                    metrics=[f1])
    return model
print('def model done')

def model done


In [12]:
from sklearn.model_selection import KFold
import gc
def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    test_pred, train_pred = np.zeros((102277,19)),np.zeros((102277,19))
#     LRDecay = LearningRateScheduler(lr_decay)
    for train_index, test_index in kf.split(train_x):
        
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        d_test = test_x
        batch_size = 64
        epochs = 15

        model = get_nn()

        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, save_best_only=True,verbose=1, monitor='val_f1',  mode='max')
        early = EarlyStopping( monitor='val_f1',  mode='max', patience=2,)
        callbacks_list = [checkpoint,early]

        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                    validation_data=(hold_out_x, hold_out_y), 
                  callbacks=callbacks_list)

        
        model.load_weights(file_path)
       
        train_pred[test_index] = model.predict(hold_out_x)
        curr_test_pred = model.predict(d_test)
        test_pred += curr_test_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    return train_pred, test_pred


print('def done')

def done


In [13]:
train_pred, nn_res = kf_train(fold_cnt=10,rnd=4)

now is 1 fold
Train on 50931 samples, validate on 5659 samples
Epoch 1/15
50931/50931 [==============================] - 12s 227us/step - loss: 0.8114 - f1: 0.7733 - val_loss: 0.7094 - val_f1: 0.8218

Epoch 00001: val_f1 improved from -inf to 0.82178, saving model to weights_base.best.h5
Epoch 2/15
50931/50931 [==============================] - 3s 68us/step - loss: 0.7063 - f1: 0.8206 - val_loss: 0.7038 - val_f1: 0.8209

Epoch 00002: val_f1 did not improve from 0.82178
Epoch 3/15
50931/50931 [==============================] - 3s 65us/step - loss: 0.6921 - f1: 0.8211 - val_loss: 0.6881 - val_f1: 0.8185

Epoch 00003: val_f1 did not improve from 0.82178
now is 2 fold
Train on 50931 samples, validate on 5659 samples
Epoch 1/15
50931/50931 [==============================] - 4s 72us/step - loss: 0.8129 - f1: 0.7725 - val_loss: 0.7068 - val_f1: 0.8207

Epoch 00001: val_f1 improved from -inf to 0.82068, saving model to weights_base.best.h5
Epoch 2/15
50931/50931 [==============================

50931/50931 [==============================] - 3s 67us/step - loss: 0.6904 - f1: 0.8207 - val_loss: 0.6963 - val_f1: 0.8204

Epoch 00003: val_f1 improved from 0.81972 to 0.82037, saving model to weights_base.best.h5
Epoch 4/15
50931/50931 [==============================] - 3s 68us/step - loss: 0.6753 - f1: 0.8208 - val_loss: 0.6808 - val_f1: 0.8188

Epoch 00004: val_f1 did not improve from 0.82037
Epoch 5/15
50931/50931 [==============================] - 3s 68us/step - loss: 0.6613 - f1: 0.8208 - val_loss: 0.6789 - val_f1: 0.8177

Epoch 00005: val_f1 did not improve from 0.82037
-------------------------------


In [15]:
name = ["class_prob_%s"%i for i in range(1,20)]
df_lgb = pd.DataFrame(nn_res, columns=name)
# df_lgb = df_lgb.drop('class_prob_20', axis=1)
df_lgb.to_csv('../pro/stacking_nn_bagging.csv', index=False)